# An introduction to the MM-GBSA method

## Background
The MM-GBSA (and related MM-PBSA) methoda are popular approches to the prediction of ligand-receptor binding free energies. They typically give better predictions than simple docking approaches, and though not as sophisticated as Free Energy Perturbation methods, are considerably less compute-intensive.

For a good review, see: [Genheden and Ryde](https://www.tandfonline.com/doi/full/10.1517/17460441.2015.1032936#d1e147).

In summary, the MM-GBSA method can be used to post=process an MD simulation of a receptor-ligand complex and extract a predicted binding free energy. The steps are:

1. Three further simulation systems are parameterized, as if for implicit solvent simulations:
    1. The complex (receptor + ligand, no explicit ions or solvent)
    2. The receptor alone.
    3. The ligand alone.

2. For each snapshot from the original simulation:
   1. The coordinates of the complex atoms alone are extracted, and the energy of this conformation, $Ec$, calculated using the implicit solvent model for the complex.
   2. The coordinates of the receptor atoms alone are extracted, and their energy $Er$ calculated using the implicit solvent model of the receptor.
   3. The coordinates of the ligand atoms alone are extracted, and their energy $El$ calculated using the implicit solvent model of the ligand.
   4. The binding energy between the receptor and ligand, $Ei$, is then given by $Ec - (Er + El)$.

4. The binding free energy is then approximated as $\langle Ei \rangle$.

![Figure 1](Figure1.png)

## MM-GBSA calculations using *Ambertools* and *OpenMM*

This worshop demonstrates how an MM-GBSA analysis can be run in a Python notebook using a combination of [OpenMM](https://openmm.org) and [parmed](https://parmed.github.io/ParmEd/html/index.html). 
The data comes from a short simulation of the complex between the protein Mcl-1 and ligand 5X3 (PDB code [5FDR](https://www.rcsb.org/structure/5FDR)), run with *OpenMM*. 

### 1. Load the required Python packages

Most of the work is done by *OpenMM*, but we also use *parmed* to help cut down the complete solvated system into complex, receptor, and ligand components, *MDTraj* for trajectory file loading and manipulation, *numpy* for some maths, and *Matplotlib* for graphs:

In [ ]:
from openmm import LangevinMiddleIntegrator

from openmm.unit import nanometer, kilojoule, mole, kelvin, picosecond
from openmm.app import HCT, NoCutoff, Simulation
from parmed.amber import AmberParm
import mdtraj as mdt
import numpy as np
from matplotlib import pyplot as plt

### 2. Load the trajectory data
In the folder you will find `5fdr_A_solvated.prmtop`, the Amber-format *prmtop* file for the system, and `5fdr_A_solvated.xtc`, the trajectory file in the compressed *Gromacs* format. We load this using *MDTraj*, then generate two lists: one of the indices of the atoms that are part of the receptor, and one for atoms that are part of the ligand. From these two we can generate a list of the atoms that form the complex:

In [ ]:
t = mdt.load('5fdr_A_solvated.xtc', top='5fdr_A_solvated.prmtop')
receptor_atoms = t.topology.select('protein')
ligand_atoms = t.topology.select('resname "5X3"')

complex_atoms = np.sort(np.concatenate((receptor_atoms, ligand_atoms)))

### 3. Generate the implicit solvent simulation systems

We can use *parmed* to manipulate the original parameter/topology file, and create simulation systems for complex, receptor, and ligand. Each is modelled with an implicit solvent model (here the Hawkins, Cramer, Truhlar model, but others are possible), and as is usual for implicit solvent simulations, no nonbonded cutoff is used:

In [ ]:
prmtop = AmberParm('5fdr_A_solvated.prmtop')

# Slice the prmtop file as required:
c_prmtop = prmtop[complex_atoms]
r_prmtop = prmtop[receptor_atoms]
l_prmtop = prmtop[ligand_atoms]

# Now generate OMM "systems":
c_system = c_prmtop.createSystem(implicitSolvent=HCT, nonbondedMethod=NoCutoff)
r_system = r_prmtop.createSystem(implicitSolvent=HCT, nonbondedMethod=NoCutoff)
l_system = l_prmtop.createSystem(implicitSolvent=HCT, nonbondedMethod=NoCutoff)

### 4. Make OMM Simulations for each System

Although we are not going to run any fresh simulations, we still need to generate *Simulation* objects for each system, so we can feed them sets of coordinates and extract from them the corresponding energy of the system. This means we have to create *Integrators* even though they will not actually be used:

In [ ]:
c_integrator = LangevinMiddleIntegrator(310*kelvin, 1/picosecond, 0.002*picosecond)
r_integrator = LangevinMiddleIntegrator(310*kelvin, 1/picosecond, 0.002*picosecond)
l_integrator = LangevinMiddleIntegrator(310*kelvin, 1/picosecond, 0.002*picosecond)
c_simulation = Simulation(c_prmtop.topology, c_system, c_integrator)
r_simulation = Simulation(r_prmtop.topology, r_system, r_integrator)
l_simulation = Simulation(l_prmtop.topology, l_system, l_integrator)

### 5. Check the systems
Before embarking on the full MM-GBSA calculation, it's good to sense-check. In the cell below we extract all the atoms that are supposed to correspond to the "complex" (receptor plus ligand) from the first snapshot of the input trajectory, feed these to the complex simulation object, query the resultant energy of the system, and check it looks reasonable: 

In [ ]:
# MDTraj stores coordinates in nanometers, but OMM needs to know this explicitly:
c_simulation.context.setPositions(t.xyz[0][complex_atoms] * nanometer)
# Query the state of the simulation:
c_state = c_simulation.context.getState(getEnergy=True)
# Print the potential energy:
print(f"Complex snapshot 0 energy: {c_state.getPotentialEnergy().format('%8.2f')}")

Hopefully the number looks reasonable. Repeat for the other two terms for this first snapshot:

In [ ]:
r_simulation.context.setPositions(t.xyz[0][receptor_atoms] * nanometer)
r_state = r_simulation.context.getState(getEnergy=True)
print(f"Receptor snapshot 0 energy: {r_state.getPotentialEnergy().format('%8.2f')}")

l_simulation.context.setPositions(t.xyz[0][ligand_atoms] * nanometer)
l_state = l_simulation.context.getState(getEnergy=True)
print(f"Ligand snapshot 0 energy: {l_state.getPotentialEnergy().format('%8.2f')}")

From these numbers, calculating $Ei$ (given by $Ec$ - ($Er$ + $El$)) should give a value of about -130 kJ/mol. If you have made an error defining which atoms belong to the receptor and/or ligand at the start of the process, you will most likely end up with nonsensical numbers instead.

### 6. Gather the MM-GBSA data for each snapshot

Assuming there is no indication of a probem, we can now run the full MM-GBSA analysis:

In [ ]:
c_energies = []
r_energies = []
l_energies = []

for i in range(t.n_frames):
    c_simulation.context.setPositions(t.xyz[i][complex_atoms] * nanometer)
    c_state = c_simulation.context.getState(getEnergy=True)
    c_energies.append(c_state.getPotentialEnergy())
        
    r_simulation.context.setPositions(t.xyz[i][receptor_atoms] * nanometer)
    r_state = r_simulation.context.getState(getEnergy=True)
    r_energies.append(r_state.getPotentialEnergy())
    
    l_simulation.context.setPositions(t.xyz[i][ligand_atoms] * nanometer)
    l_state = l_simulation.context.getState(getEnergy=True)
    l_energies.append(l_state.getPotentialEnergy())

### 7. Process the data to estimate the binding free energy

We now have three list of energy components per snapshot. It's convenient to convert these to *numpy* arrays for processing:

In [ ]:
c_energies = np.array(c_energies)
r_energies = np.array(r_energies)
l_energies = np.array(l_energies)

interaction_energies = c_energies - (r_energies + l_energies)
print(f"MM-GBSA prediction of binding affinity = {interaction_energies.mean().format('%8.2f')} SD: {interaction_energies.std():5.2f}")

### 8. Explore the data in more detail

To dig a bit deeper into the numbers, a graph can help. Let's plot the individual *Ei* values for each snapshot. Not that *Matplotlib* doesn't understand *OpenMM*'s concept of *Quantities* so we have to strip these off before plotting:

In [ ]:
energy_unit = interaction_energies[0].unit
inverse_unit = np.array(1/energy_unit)
plt.plot(t.time, interaction_energies * inverse_unit)
plt.ylabel(f'Interaction Energy ({energy_unit.get_symbol()})')
plt.xlabel('time (ps)')

We can see the first two or three points might be outliers - hardly surprising in this case as the data comes from a very short simulation of a system that begun from the crystal structure conformation and so was almost certainly not equilibrated/relaxed. A "real" MM-GBSA calculation needs an equilbrated, well-sampled, trajectory to work with.

## Summary

This workshop demonstrates a quick and easy approach to performing an MM-GBSA analysis of a simulation. It is possible to be much more sophisticated than this, for example to extract individual energy terms that contribute to the overall binding free energy, or to make a more complete analysis of entropic contributions to the free energy. 

See the suggested [review](https://www.tandfonline.com/doi/full/10.1517/17460441.2015.1032936#d1e147) for more on this.